In [39]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import load_model
from functools import partial
from numpy import array
import pandas as pd
import numpy as np

In [67]:
# input data
data = pd.read_csv("output.csv", index_col=0)


X_train = data.iloc[:, :-2]
y_train = data.iloc[:, -2].reshape((y_train.shape[0],1))

In [77]:
# set parameters
nSample = X_train.shape[0]
timestep = X_train.shape[1]
features = 1

In [78]:
timestep

240

In [70]:
x = x.reshape((nSample, timestep, features))

In [71]:
x.shape

(5100, 240, 1)

In [97]:
# define model
# an input layer that expects 1 or more samples, 1 time steps, and 1 feature
model = Sequential()
model.add(LSTM(20, activation='tanh', recurrent_activation='tanh',
               input_shape=(timestep, features), recurrent_dropout = 0.1))
model.add(Dense(1, activation='sigmoid'))

# compile model
model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

# checkpoints & earlystopping
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True,
                             save_weights_only=True, mode='min', period=1)
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')
callbacks_list = [checkpoint, earlystop]

In [98]:
# fit model
history = model.fit(x, y_train, epochs=1000, validation_split=0.2,
                callbacks= callbacks_list, shuffle=False, verbose=1)

Train on 4080 samples, validate on 1020 samples
Epoch 1/1000
4080/4080 [==============================] - 24s 6ms/step - loss: 0.6925 - acc: 0.5275 - val_loss: 0.6940 - val_acc: 0.4922
Epoch 2/1000
2944/4080 [====================>.........] - ETA: 5s - loss: 0.6920 - acc: 0.5251

KeyboardInterrupt: 

In [82]:
y_train.sum()/len(y_train)

0.52

In [ ]:
# evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print ('loss = ', loss, '\naccuracy =', accuracy)

# save model to single file
model.save('lstm_model.h5')

In [20]:
# load model from single file
model = load_model('lstm_model.h5')

# make predictions
y_hat = model.predict_classes(X_test,verbose=0)
y_prob = model.predict_proba(X_test,verbose=0)
#print(y_hat)
#print(y_prob)